In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'fourier'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_fourier_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,15,16,17,18,19,20,21,22,23,24
2,1179.27,-0.01,0.01,0.02,0.00,0.00,0.01,0.03,-0.00,0.01,...,0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.0,-0.00
3,-0.01,807.32,-0.00,-0.01,0.01,-0.02,0.00,0.01,-0.00,-0.00,...,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,-0.0,-0.00
4,0.01,-0.00,609.13,0.02,0.01,-0.00,0.01,-0.01,0.01,0.00,...,-0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.0,0.00
5,0.02,-0.01,0.02,437.01,-0.02,0.02,-0.00,-0.00,0.00,0.00,...,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.0,0.00
6,0.00,0.01,0.01,-0.02,298.32,-0.00,-0.00,-0.00,-0.00,0.00,...,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,-0.0,-0.00
7,0.00,-0.02,-0.00,0.02,-0.00,188.41,-0.00,0.00,0.00,-0.00,...,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.0,-0.00
8,0.01,0.00,0.01,-0.00,-0.00,-0.00,126.23,0.00,-0.00,0.00,...,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.0,-0.00
9,0.03,0.01,-0.01,-0.00,-0.00,0.00,0.00,76.41,-0.00,-0.00,...,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.0,-0.00
10,-0.00,-0.00,0.01,0.00,-0.00,0.00,-0.00,-0.00,50.90,0.00,...,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,0.0,0.00
11,0.01,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,33.34,...,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.0,0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,15,16,17,18,19,20,21,22,23,24
2,1.00000,-0.00001,0.00001,0.00003,0.00001,0.00001,0.00004,0.00010,-0.00001,0.00003,...,0.00001,0.00001,-0.00002,0.00007,0.00005,-0.00000,0.00002,-0.00001,0.00004,-0.00002
3,-0.00001,1.00000,-0.00000,-0.00002,0.00003,-0.00006,0.00001,0.00003,-0.00002,-0.00002,...,-0.00001,0.00004,0.00000,0.00003,0.00003,-0.00001,0.00003,-0.00004,-0.00004,-0.00001
4,0.00001,-0.00000,1.00000,0.00004,0.00002,-0.00000,0.00003,-0.00004,0.00007,0.00000,...,-0.00004,0.00001,0.00003,-0.00004,0.00001,0.00001,-0.00000,-0.00002,0.00002,0.00000
5,0.00003,-0.00002,0.00004,1.00000,-0.00004,0.00005,-0.00001,-0.00003,0.00001,0.00001,...,-0.00002,-0.00002,0.00001,-0.00001,0.00004,0.00000,0.00000,-0.00002,0.00005,0.00003
6,0.00001,0.00003,0.00002,-0.00004,1.00000,-0.00000,-0.00001,-0.00000,-0.00004,0.00003,...,0.00002,-0.00001,0.00002,0.00000,0.00003,0.00004,-0.00002,0.00002,-0.00004,-0.00002
7,0.00001,-0.00006,-0.00000,0.00005,-0.00000,1.00000,-0.00000,0.00000,0.00003,-0.00002,...,-0.00004,0.00002,-0.00003,0.00008,-0.00003,0.00004,0.00002,-0.00005,0.00000,-0.00002
8,0.00004,0.00001,0.00003,-0.00001,-0.00001,-0.00000,1.00000,0.00001,-0.00005,0.00005,...,0.00001,0.00003,-0.00001,0.00000,-0.00000,0.00000,-0.00002,-0.00007,-0.00003,-0.00000
9,0.00010,0.00003,-0.00004,-0.00003,-0.00000,0.00000,0.00001,1.00000,-0.00003,-0.00002,...,0.00001,0.00000,-0.00001,0.00005,-0.00002,0.00003,0.00001,0.00002,-0.00003,-0.00006
10,-0.00001,-0.00002,0.00007,0.00001,-0.00004,0.00003,-0.00005,-0.00003,1.00000,0.00006,...,0.00001,-0.00001,0.00000,-0.00002,0.00001,0.00003,-0.00003,0.00001,0.00001,0.00002
11,0.00003,-0.00002,0.00000,0.00001,0.00003,-0.00002,0.00005,-0.00002,0.00006,1.00000,...,0.00001,0.00001,0.00005,-0.00001,0.00004,0.00006,-0.00004,-0.00001,-0.00006,0.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.09604775117056488

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[6.06454734e+04 2.83577438e+04 1.60688062e+04 8.24398572e+03
 3.82992588e+03 1.52521519e+03 6.81373417e+02 2.49298093e+02
 1.10072474e+02 4.70170949e+01 2.13236857e+01 9.78018118e+00
 4.44631140e+00 1.81278022e+00 7.33340524e-01 2.59017276e-01
 8.83896984e-02 2.32731125e-02 6.79471833e-03 1.60662156e-03
 6.66917730e-04 3.38535419e-04 8.72776790e-35]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,0.997670,-0.056550,-0.030198,-0.018235,-0.011444,-0.006927,-0.004562,-0.002721,-0.001828,-0.001189,...,-0.000237,-0.000152,-0.000092,-0.000052,-0.000027,-0.000015,-0.000007,-0.000005,-0.000003,-0.000003
1,0.052867,0.994272,-0.082778,-0.034721,-0.019135,-0.011037,-0.007128,-0.004234,-0.002821,-0.001842,...,-0.000367,-0.000232,-0.000140,-0.000081,-0.000042,-0.000024,-0.000011,-0.000008,-0.000006,-0.000004
2,0.032465,0.077318,0.992766,-0.076662,-0.032183,-0.016932,-0.010661,-0.006292,-0.004122,-0.002703,...,-0.000542,-0.000343,-0.000204,-0.000124,-0.000063,-0.000035,-0.000017,-0.000011,-0.000008,-0.000006
3,0.021277,0.037530,0.070042,0.993483,-0.070667,-0.028155,-0.016670,-0.009523,-0.006213,-0.004040,...,-0.000804,-0.000515,-0.000306,-0.000182,-0.000092,-0.000052,-0.000025,-0.000017,-0.000011,-0.000009
4,0.014246,0.022431,0.033503,0.064750,0.994486,-0.060993,-0.029168,-0.015384,-0.009857,-0.006316,...,-0.001244,-0.000799,-0.000475,-0.000283,-0.000145,-0.000079,-0.000040,-0.000025,-0.000019,-0.000014
5,0.008912,0.013431,0.018516,0.028417,0.055204,0.994414,-0.073488,-0.028277,-0.016842,-0.010551,...,-0.002045,-0.001300,-0.000783,-0.000454,-0.000240,-0.000129,-0.000063,-0.000042,-0.000031,-0.000023
6,0.006356,0.009415,0.012688,0.018446,0.029950,0.068249,0.994225,-0.061999,-0.030551,-0.017804,...,-0.003305,-0.002107,-0.001266,-0.000746,-0.000387,-0.000213,-0.000104,-0.000069,-0.000051,-0.000037
7,0.003889,0.005737,0.007683,0.010851,0.016436,0.028876,0.055557,0.993501,-0.081034,-0.035095,...,-0.005622,-0.003584,-0.002144,-0.001257,-0.000657,-0.000359,-0.000175,-0.000112,-0.000085,-0.000064
8,0.002806,0.004112,0.005439,0.007653,0.011406,0.018798,0.030923,0.073958,0.991731,-0.083919,...,-0.009241,-0.005858,-0.003489,-0.002060,-0.001064,-0.000585,-0.000286,-0.000183,-0.000136,-0.000100
9,0.001933,0.002838,0.003760,0.005254,0.007739,0.012496,0.019324,0.036372,0.074591,0.989953,...,-0.015342,-0.009574,-0.005653,-0.003337,-0.001717,-0.000943,-0.000465,-0.000299,-0.000225,-0.000163


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0023304072706630263,
 0.005728162496796307,
 0.007233653424549291,
 0.00651652901254407,
 0.005514340807426521,
 0.005586112080493311,
 0.005775183188940658,
 0.006498808631132946,
 0.008268954572147935,
 0.010046619024545067,
 0.012571078599038499,
 0.01487589903434361,
 0.01583342981761393,
 0.016694611225011702,
 0.017708090916657815,
 0.01868319600198809,
 0.01918266203831198,
 0.02278871505665947,
 0.030261851746602786,
 0.06564338453672836,
 0.1586799323047049,
 0.31197623072293335,
 0.29519602873534156]